In [ ]:
#import pandas as pd
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go

import experiment_helpers

#debug
from importlib import reload
from experiment_helpers import *
reload(experiment_helpers)
from experiment_helpers import *

In [ ]:
EXPERIMENT_TYPES = [
    "no_prefetching",
    "linux_prefetching",
    #"linux_prefetching_ssdopt",
    #"linux_prefetching_ssdopt_asyncwrites",
    #"linux_prefetching_ssdopt_asyncwrites",

    "tape_prefetching_syncwrites",
    "tape_prefetching_asyncwrites",
    "tape_prefetching_asyncwrites_linux",

    #"tape_prefetching_asyncwrites_offload_fetch",
]
annotations=[]

In [ ]:
def add_annotation(text):
    anno = "<h3><pre>%s</pre></h3>" % text
    annotations.append(anno)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Experiment plots

## Define experiments

In [ ]:
WORKLOADS = ["mmult_eigen",#"vec_eigen",
             #"dot_eigen", "mmap_random_rw","kissfft",  "kmeans", "torch",
            # "bitonic_merge", "native_sort",
            # "bitonic_sort", "torch", "torch_par","torch_par4","linpack", "mmult_eigen_par"
            ]
#"alexnet", "vgg16", "batcher_sort"]:

## Load data(single pandas table of all experiments per workload)

In [ ]:
data = {}
for workload in WORKLOADS:
    data[workload] = augment_tables(get_experiment_data(EXPERIMENT_TYPES, workload,"../experiment_results"), filter_raw=True)

## Explore & Plot

In [ ]:
## TODO:::take_column_named:: VERY VERY HACKY.. assumes all tables have data appropriately sorted
## sanity check later with more data, if these plots become crucial
runtime = take_column_named("Measured(wallclock) runtime",data).dropna()
fig = px.line(runtime, title='Absolute(Wallclock) Runtime vs. Local Memory', 
              animation_frame="Experiment Name")
fig.update_layout(
    xaxis_title="Ratio(%)",
    yaxis_title="Measured(wallclock) runtime(s)",
)
fig.show()

degradations = take_column_named("Degradation(%)",data).dropna()
fig = px.line(degradations, title='Normalized Runtime(Wallclock) vs. Local Memory',
                animation_frame="Experiment Name")

fig.update_layout(
    xaxis_title="Ratio(%)",
    yaxis_title="Degradation(%)",
)
fig.show()

degradations_no_ev = take_column_named("Degradation w/o Evictions(%)",data).dropna()
fig = px.line(degradations_no_ev, title='Normalized Runtime vs. Local Memory (SUBTRACT time spent in evictions)',
             animation_frame="Experiment Name")
fig.update_layout(
    xaxis_title="Ratio(%)",
    yaxis_title="Degradation w/o Evictions(%)",
)
fig.show()

In [ ]:
runtime_components = []

for workload in WORKLOADS:
    fig = get_components_of_runtime(data[workload], "%s" % workload)
    runtime_components.append(fig)
    fig.update_layout(yaxis_range=[0,15])
    #with open('figs/comp_%s_%s.html' % (exp_type, workload), 'w') as f:
    #    f.write(fig.to_html(full_html=False, include_plotlyjs='plotly-latest.min.js'))

    fig.show()

In [ ]:
pd.read_csv("../experiment_results/mmult_eigen/tape_prefetching_syncwrites/nic_monitor.70.csv")


In [ ]:
go.Line(pd.read_csv("../experiment_results/mmult_eigen/tape_prefetching_syncwrites/nic_monitor.30.csv"), x='TIME', y=['XMIT','RECV'])


In [ ]:
d

In [ ]:

d = pd.read_csv("../experiment_results/mmult_eigen/no_prefetching/nic_monitor.30.csv")
d["RATIO"] = 30
d2 = pd.read_csv("../experiment_results/mmult_eigen/no_prefetching/nic_monitor.20.csv")
d2["RATIO"] = 20
d = d.append(d2)
px.line(d, x='TIME', y=['XMIT','RECV'], animation_frame="RATIO")


In [ ]:
t = data["no_prefetching"]["mmult_eigen"]
t["EXP"] = "no_prefetching"
t2 = data["linux_prefetching"]["mmult_eigen"]
t2["EXP"] = "linux_prefetching"

t = t.append(t2)

In [ ]:
#t=data["no_prefetching"]["mmult_eigen"]
px.area(t[["Eviction Time",
                         "Baseline minor PF Time",
                         "Extra Minor PF Time",
                         "Major PF Time",
                         "Baseline User Time",
                         "Extra User Time", "EXP"
                                         ]],
                  title='Components of runtime(%s)'%'aa',
                  color_discrete_sequence=['#636efa', '#ef553b',  '#9e1700','#00cc96', '#ab63fa', '#3c0c73'],
       animation_frame="EXP")

In [ ]:
(exp_type, workload) = "linux_prefetching", "torch_par"
get_components_of_runtime(data[exp_type][workload], "%s,%s, +4us rdma delay" % (exp_type, workload))

In [ ]:
data['linux_prefetching']['bitonic_sort']['Minor Page Faults']/ \
data['linux_prefetching']['bitonic_sort']['Major Page Faults']

In [ ]:
for exp_name in ["tape_prefetching_syncwrites", "tape_prefetching_asyncwrites"]:
    fig = get_components_of_runtime(get_experiment_table(named_dfs,exp_name))

    runtime_components.append(fig)
    fig.update_layout(yaxis_range=[0,15])
    fig.show()


In [ ]:
def spill_html():
    with open('linux_delay_8.html', 'w') as f:
        i = 0
        for figures in [plots,runtime_components]:
            for figure in figures:
                #anno = annotations[i]
                #i+=1
                f.write(figure.to_html(full_html=False, include_plotlyjs='cdn'))
                #f.write(anno)
                f.write("<hr/>")